# CatBoost Regressor

## Mục tiêu

- Xây dựng mô hình CatBoost Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation

## Giới thiệu

CatBoost là một biến thể gradient boosting trên Decision Tree, được thiết kế để xử lý biến phân loại một cách thông minh và có khả năng giảm overfitting tốt hơn các thuật toán như XGBoost, LightGBM.

### **Ưu điểm:**
- Xử lý dữ liệu phân loại mà không cần one-hot encoding.
- Chống overfitting tốt.
- Hỗ trợ xử lý missing values tự động.
- Ổn định và mạnh cho dữ liệu nhỏ.

### **Nhược điểm:**
- Model nặng, đặc biệt khi dùng nhiều cây.
- Thời gian training có thể rất chậm và tốn RAM khi xử lý dữ liệu lớn.


## Bước 1 - Import các thư viện cần thiết

### 1.1. Cài thư viện catboost

In [ ]:
!pip install catboost

### 1.2. Import các thư viện cơ bản khác

In [1]:
import pandas as pd
import os
import joblib
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

## Bước 2 - Khai báo mô hình
Đọc dữ liệu từ các file CSV đã được tiền xử lý.

In [2]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3255, 13)

5 dòng đầu tiên của tập train (đã được chuẩn hóa):


,country_name,country_code,year,population,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,labor_force
0,Romania,ROU,2019,-0.100387,-1.101137,75.607317,-0.107887,0.079336,1.170060,0.621932,0.746494,-0.047183,-0.871164
1,Mauritius,MUS,2006,-0.235882,-0.500838,72.432195,-0.405891,0.241659,-0.060699,0.586683,0.718065,-0.221970,-0.189257
2,Angola,AGO,2008,-0.083899,1.500859,55.281000,-0.470953,1.327705,-0.060699,-1.545881,-2.268636,-0.432669,1.598947
3,Albania,ALB,2001,-0.222240,-1.349940,75.083000,-0.584364,0.930412,-1.071878,0.600783,-0.024271,-0.404910,-0.104497
4,Central African Republic,CAF,2021,-0.206911,0.236338,40.279000,-0.618434,-0.427187,-1.640089,-2.349558,-3.008624,-0.529029,1.172890


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`

In [3]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 10

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. pop_growth
  4. gdp_per_capita
  5. gdp_growth
  6. sanitation
  7. electricity
  8. water_access
  9. co2_emissions
  10. labor_force

Kích thước X_train: (3255, 10)
Kích thước y_train: (3255,)


## Bước 4 - Xây dựng và huấn luyện mô hình CatBoost

### 4.1. Sử dụng RandomizedSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình CatBoost

In [ ]:
from scipy.stats import loguniform, randint, uniform
import shutil

# Không gian tìm kiếm siêu tham số
param_distributions_catboost = {
    "depth": randint(4, 11),                      
    "learning_rate": loguniform(1e-3, 3e-1),      
    "n_estimators": randint(300, 1500),          
    "l2_leaf_reg": loguniform(1e-1, 10),         
    "subsample": uniform(0.6, 0.4),         
    "colsample_bylevel": uniform(0.5, 0.5),     
    "min_data_in_leaf": randint(1, 50),         
    "random_strength": uniform(0.5, 2.0),         
}

# Khai báo mô hình CatBoost với train_dir để tránh lỗi tmp
base_model = CatBoostRegressor(
    loss_function="RMSE",
    random_state=42,
    verbose=0,
)

# Tạo RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions_catboost,
    n_iter=60,
    scoring="neg_root_mean_squared_error",
    cv=5,          
    n_jobs=-1,
    random_state=42,
    verbose=2
)

# Thực hiện tìm kiếm
random_search.fit(X_train, y_train)


Fitting 5 folds for each of 60 candidates, totalling 300 fits


### 4.2. Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [ ]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_catboost = random_search.best_estimator_
best_params = random_search.best_params_

print("="*60)
print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

### 4.3. - Đánh giá mô hình tối ưu trên tập train

In [ ]:
# Dự đoán trên tập train và validation
y_train_pred = best_catboost.predict(X_train)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: CatBoost")
print(f"RMSE loss: {train_rmse:.3f}")
print(f"MAE loss: {train_mae:.3f}")
print(f"R2 score: {train_r2:.3f}")

### 4.4. Lưu mô hình Cat Boost đã tối ưu vào file để sử dụng sau này.

In [ ]:
# Tạo thư mục saved_models nếu chưa có
os.makedirs('../models/8_catboost', exist_ok=True)

# Lưu mô hình
model_path = '../models/8_catboost/catboost.pkl.gz'
joblib.dump(best_catboost, model_path, compress=('gzip', 9))

## Kết luận

### Tổng kết:
1. Mô hình CatBoost đã được xây dựng và tối ưu hóa thành công
2. RandomizedSearchCV với 5-Fold CV đã tìm được bộ siêu tham số tối ưu
3. Mô hình được đánh giá dựa trên 3 metrics: MAE, RMSE và R2